In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import time
from sklearn.model_selection import train_test_split
tf.logging.set_verbosity(tf.logging.INFO)

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Convert classes to indicator vectors
def one_hot(values,n_values=51):
    n_v = np.maximum(n_values, np.max(values) + 1)
    oh = np.eye(n_v)[values]
    return oh

In [0]:
frame_data = np.float16(np.load('drive/My Drive/Computer Vision/HMDB/saved_training/frame_data_10_frame_flat_1544666342.npy'))
labels = np.load('drive/My Drive/Computer Vision/HMDB/saved_training/labels_10_frame_1544666342.npy')

n = 0
class_labels = {}
new_labels = []
for l in labels:
  if l in class_labels:
    new_labels.append(class_labels[l])
  else:
    class_labels[l] = n
    new_labels.append(n)
    n += 1
labels = one_hot(np.array(new_labels))

train_data, test_data, train_labels, test_labels = train_test_split(frame_data, labels, test_size=0.1, random_state=102)
val_data = train_data[int(0.9 * len(train_data)):len(train_data)]
val_labels = np.int16(train_labels[int(0.9 * len(train_data)):len(train_labels)])
train_data = train_data[0:int(0.9 * len(train_data))]
train_labels = np.int16(train_labels[0:int(0.9 * len(train_labels))])

In [11]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(125,125,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

# model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
# model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(51, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 125, 125, 64)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 62, 62, 32)        8224      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 31, 31, 32)        4128      
__________

In [12]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
train_data = train_data.reshape(-1,125,125,1)
val_data = val_data.reshape(-1, 125, 125, 1)
model.fit(train_data, train_labels, batch_size=100, epochs=15, validation_data=(val_data, val_labels))

Train on 55096 samples, validate on 6122 samples
Epoch 1/15
55096/55096 [==============================] - 80s 1ms/step - loss: 3.4684 - acc: 0.1286 - val_loss: 3.0301 - val_acc: 0.2463
Epoch 2/15
55096/55096 [==============================] - 79s 1ms/step - loss: 2.6787 - acc: 0.2960 - val_loss: 2.3293 - val_acc: 0.4167
Epoch 3/15
55096/55096 [==============================] - 79s 1ms/step - loss: 2.1763 - acc: 0.4107 - val_loss: 1.8733 - val_acc: 0.5198
Epoch 4/15
55096/55096 [==============================] - 79s 1ms/step - loss: 1.8803 - acc: 0.4848 - val_loss: 1.5640 - val_acc: 0.5967
Epoch 5/15
55096/55096 [==============================] - 79s 1ms/step - loss: 1.6739 - acc: 0.5343 - val_loss: 1.3732 - val_acc: 0.6457
Epoch 6/15
55096/55096 [==============================] - 78s 1ms/step - loss: 1.5186 - acc: 0.5730 - val_loss: 1.2653 - val_acc: 0.6689
Epoch 7/15
55096/55096 [==============================] - 79s 1ms/step - loss: 1.4000 - acc: 0.6022 - val_loss: 1.1291 - val_acc:

In [13]:
test_data = test_data.reshape(-1, 125, 125, 1)
score = model.evaluate(test_data, test_labels)
print('\n', 'Test accuracy:', score[1])

6802/6802 [==============================] - 4s 591us/step

 Test accuracy: 0.8306380475629466
